### Import of necessary libraries

In [1]:
from interface import Interface
from shapely import Polygon
import folium
import geopandas

### Creation of a polygon as an example of AOI

In [2]:
# simplified example of a call to the interface

lat_point_list = [50.854457, 50.924457, 52.518172, 50.072651, 48.853033]
lon_point_list = [4.377184, 4.377184, 13.407759, 14.435935, 2.349553]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))

polygon_geom = geopandas.GeoSeries(polygon_geom)

m = folium.Map(location=[50.854457, 4.377184], zoom_start=5)
folium.GeoJson(polygon_geom.__geo_interface__).add_to(m)

x = geopandas.GeoSeries(polygon_geom)
              
rectangle = geopandas.GeoDataFrame(geometry=x.envelope)

# Define the number of rows and columns for the grid
num_rows = 5
num_cols = 5

# Calculate the width and height of each sub-area
width = (rectangle.bounds['maxx'] - rectangle.bounds['minx']) / num_cols
height = (rectangle.bounds['maxy'] - rectangle.bounds['miny']) / num_rows

# Create an empty list to store the sub-areas
sub_areas = []

# Iterate over the rows and columns to create the grid of sub-areas
for row in range(num_rows):
    for col in range(num_cols):
        minx = rectangle.bounds['minx'] + col * width
        maxx = minx + width
        miny = rectangle.bounds['miny'] + row * height
        maxy = miny + height
        sub_area_coords = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy)]
        sub_areas.append(Polygon(sub_area_coords))

# Create a GeoDataFrame with the sub-areas
sub_areas_gdf = geopandas.GeoDataFrame(geometry=sub_areas)

# Perform the spatial intersection to obtain the divided sub-areas
divided_areas = geopandas.overlay(sub_areas_gdf, rectangle, how='intersection')

# print(divided_areas)

for index, row in divided_areas.iterrows():
    geojson = row.geometry.__geo_interface__
    folium.GeoJson(geojson).add_to(m)
    
centroids = set()

for index, row in divided_areas.iterrows():    
    if(polygon_geom.intersects(row.geometry).bool()==True):
        centroids.add(row.geometry.centroid)

for centroid in centroids:
    print(centroid)
    folium.GeoJson(centroid).add_to(m)

# folium.GeoJson(divided_areas.to_json).add_to(m)
   
m

POINT (8.392744 49.95257470000001)
POINT (5.975467600000001 49.219546900000005)
POINT (13.227296800000001 51.418630300000004)
POINT (13.227296800000001 49.95257470000001)
POINT (3.5581912 49.95257470000001)
POINT (8.392744 50.685602500000016)
POINT (8.392744 52.151658100000006)
POINT (10.8100204 50.685602500000016)
POINT (3.5581912 50.6856025)
POINT (13.227296800000001 52.151658100000006)
POINT (10.8100204 49.219546900000005)
POINT (5.975467600000001 51.41863030000001)
POINT (3.5581912 49.2195469)
POINT (8.392744 49.219546900000005)
POINT (5.975467600000001 49.9525747)
POINT (10.8100204 51.418630300000004)
POINT (5.975467600000001 50.68560250000001)
POINT (10.8100204 49.95257470000001)
POINT (8.392744 51.418630300000004)
POINT (10.8100204 52.151658100000006)
POINT (13.227296800000001 50.685602500000016)


### The class interface is created and the method get_weather_forecast is called by group 6

In [3]:
interface = Interface()
interface.get_weather_forecast(polygon_geom)

{'u3', 'u2', 'u1', 'u0'}


,time,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,...,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm,soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm,is_day,AOI_ID,EventID
0,2023-05-16T00:00,10.3,95,9.5,8.1,45,0.0,0.0,0.0,0.0,...,11.8,11.6,0.111,0.105,0.071,0.079,0.114,0,1,1
1,2023-05-16T01:00,10.2,93,9.1,7.6,34,0.0,0.0,0.0,0.0,...,11.6,11.6,0.109,0.103,0.073,0.079,0.114,0,1,1
2,2023-05-16T02:00,10.1,91,8.6,7.8,24,0.0,0.0,0.0,0.0,...,11.5,11.5,0.105,0.102,0.074,0.079,0.114,0,1,1
3,2023-05-16T03:00,10.4,92,9.1,8.1,13,0.0,0.0,0.0,0.0,...,11.4,11.5,0.109,0.107,0.086,0.082,0.117,1,1,1
4,2023-05-16T04:00,10.7,90,9.2,8.5,11,0.0,0.0,0.0,0.0,...,11.3,11.5,0.107,0.105,0.087,0.082,0.117,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2023-05-22T19:00,20.3,58,11.7,19.7,24,0.0,0.0,0.0,0.0,...,17.6,12.4,0.018,0.037,0.046,0.063,0.105,0,1,1
164,2023-05-22T20:00,18.5,67,12.1,18.1,18,0.0,0.0,0.0,0.0,...,17.3,12.5,0.019,0.036,0.046,0.063,0.105,0,1,1
165,2023-05-22T21:00,16.4,77,12.4,16.1,13,0.0,0.0,0.0,0.0,...,16.9,12.5,0.020,0.036,0.046,0.063,0.105,0,1,1
166,2023-05-22T22:00,15.3,83,12.5,15.1,13,0.0,0.0,0.0,0.0,...,16.5,12.6,0.021,0.036,0.046,0.063,0.105,0,1,1
